In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch
import os
import pandas as pd
import cv2
import requests
# from torchviz import make_dot
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
test_df = pd.read_csv("/home/vishwa/amzn-ml/student_resource 3/dataset/test.csv")

test_df

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


In [5]:
completed = pd.read_csv('/home/vishwa/amzn-ml/completed.csv')
completed

,index,prediction
0,0,75.0 centimetre
1,1,6.0 centimetre
2,2,18.0 centimetre
3,3,200.0 centimetre
4,4,4.0 centimetre
...,...,...
131182,131283,1.5 pound
131183,131284,1.0 ton
131184,131285,5.0 ton
131185,131286,55.0 gram


In [ ]:
test_df.

In [9]:
new = test_df.merge(completed, on='index', how='outer')

In [13]:
new = new.loc[:,["prediction"]]
new.to_csv('completed3.csv', index=True)

In [14]:
new

,prediction
0,75.0 centimetre
1,6.0 centimetre
2,18.0 centimetre
3,200.0 centimetre
4,4.0 centimetre
...,...
131182,1.5 pound
131183,1.0 ton
131184,5.0 ton
131185,55.0 gram


In [7]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [8]:
from torch.utils.data import Dataset
# from datasets import load_dataset

class TestDataset(Dataset):
    def __init__(self, train_df):
        self.data = train_df.loc[: train_df.shape[0] // 2]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # print(idx)
        sample = self.data.loc[idx, :]
        # print(sample)
        return {
            "image": Image.open('/home/vishwa/amzn-ml/student_resource 3/test_images/' + sample['image_link'].split('/')[-1]), # Should be a PIL image
            "qa": [
                {
                    "question": f"Extract the {sample['entity_name']} from the text and only use these units {entity_unit_map[sample['entity_name']]}",
                }
            ]
        }

datasets = {
    "test": TestDataset(test_df)
}

In [9]:
# moon dream 2
import torch
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

DEVICE = "cuda"
DTYPE = torch.float32 if DEVICE == "cpu" else torch.float16 # CPU doesn't support float16
MD_REVISION = "2024-07-23"

tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2", revision=MD_REVISION)

USE_LORA = True
USE_QLORA = True

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules='.*(text_model|vision_encoder).*(out_proj|Wqkv|fc1|fc2|qkv).*$',
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )

    moondream = AutoModelForCausalLM.from_pretrained(
    "vikhyatk/moondream2", revision=MD_REVISION, trust_remote_code=True, 
    attn_implementation="flash_attention_2" if DEVICE == "cuda" else None,
    torch_dtype=torch.float16, device_map={"": DEVICE}
)
    
    # moondream.enable_input_requires_grad()
    # moondream.add_adapter(lora_config)
    # moondream.enable_adapters()

    from peft import get_peft_model

moondream = get_peft_model(moondream, lora_config)
moondream.print_trainable_parameters()

trainable params: 9,809,664 || all params: 1,876,729,456 || trainable%: 0.5227


In [10]:
moondream.load_state_dict(torch.load('/home/vishwa/amzn-ml/final.pth'))

<All keys matched successfully>

In [12]:
from tqdm import tqdm

In [13]:
moondream.eval()
test_df['pred'] = ''
for i in tqdm(range(len(test_df))):
    sample = datasets['test'][i]
    # display(sample['image'])
    # print(sample['image'])
    for qa in sample['qa']:
        # print("Question", qa['question'])
        # print("Answer", qa['answer'])
        # print(moondream.vision_encoder.preprocess(sample['image']).shape)

        preprocessed_image = moondream.vision_encoder.preprocess(sample['image'])
        output = moondream.answer_question(moondream.encode_image(preprocessed_image.reshape(1, preprocessed_image.shape[0],preprocessed_image.shape[1], preprocessed_image.shape[2])), qa['question'],tokenizer = tokenizer)
        test_df['pred'].loc[i] = output

  0%|          | 0/131187 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
/tmp/ipykernel_469584/2219626337.py:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step a

KeyboardInterrupt: 